In [ ]:
from os import listdir, path
import re

import time
import numpy as np
import random
import math

In [ ]:
BASE_DIR_PATH = path.dirname(path.abspath("__file__"))  # gdyby nie działało, usuń cudzysłów
INSTANCES_PATH = path.join(BASE_DIR_PATH, "instances")
NEGATIVE_RANDOM_PATH = path.join(INSTANCES_PATH, "negative_random")
NEGATIVE_REPETITIONS_PATH = path.join(INSTANCES_PATH, "negative_repetitions")
POSITIVE_END_ERRORS_PATH = path.join(INSTANCES_PATH, "positive_end_errors")
POSITIVE_RANDOM_PATH = path.join(INSTANCES_PATH, "positive_random")

In [ ]:
negative_random_filenames = [path.join(NEGATIVE_RANDOM_PATH, f) for f in listdir(NEGATIVE_RANDOM_PATH) if path.isfile(path.join(NEGATIVE_RANDOM_PATH, f))]
negative_repetitions_filenames = [path.join(NEGATIVE_REPETITIONS_PATH, f) for f in listdir(NEGATIVE_REPETITIONS_PATH) if path.isfile(path.join(NEGATIVE_REPETITIONS_PATH, f))]
positive_end_errors_filenames = [path.join(POSITIVE_END_ERRORS_PATH, f) for f in listdir(POSITIVE_END_ERRORS_PATH) if path.isfile(path.join(POSITIVE_END_ERRORS_PATH, f))]
positive_random_filenames = [path.join(POSITIVE_RANDOM_PATH, f) for f in listdir(POSITIVE_RANDOM_PATH) if path.isfile(path.join(POSITIVE_RANDOM_PATH, f))]

selected_filenames = negative_random_filenames + negative_repetitions_filenames + positive_end_errors_filenames + positive_random_filenames

In [ ]:
for filename in selected_filenames:
    with open(filename, 'r') as file:
        inst_name = filename[filename.rfind('/') + 1:]
        
        words = file.read().splitlines()
        
        word_len = len(words[0])
        org_words_num = int(re.search('(?<=\.)[0-9]+', inst_name).group(0))
        org_seq_len = org_words_num + word_len - 1
        neg_errors_number = re.search('(?<=\-)[0-9]+', inst_name)
        neg_errors_number = int(neg_errors_number.group(0)) if neg_errors_number else 0
        pos_errors_number = re.search('(?<=\+)[0-9]+', inst_name)
        pos_errors_number = int(pos_errors_number.group(0)) if pos_errors_number else 0
        
        print("nazwa pliku: " + inst_name)
        print("długość słowa: " + str(word_len))
        print("oryginalna liczba słów: " + str(org_words_num))
        print("oryginalna długość sekwencji: "+ str(org_seq_len))
        print("liczba błędów negatywnych: " + str(neg_errors_number))
        print("liczba błędów pozytywnych: " + str(pos_errors_number))
        print(words[:5])
        print("\n")

In [ ]:
#funkcja zwraca maksymalną liczbę nukleotydów, na których zazębia się sufiks nucl1 z prefiksem nucl2
#poszukiwanie poprzez coraz to mniejszą długość ciągu, porównywanie ciągów znak po znaku
def common_nucleotides_max_number_full_iterating(nucl1, nucl2):
    word_len = len(nucl1)
    result = word_len - 1       #zmienna zawierająca końcowy wynik
    
    while(result > 0):          #badanie coraz to mniejszej możliwej długości wspólnego ciągu
        i1 = word_len - result
        i2 = 0
        
        while(i1 < word_len):              #porównanie nachodzących końcówek, znak po znaku
            if(nucl1[i1] != nucl2[i2]):    #znaki się nie zgadzają
                break
            i1 += 1
            i2 += 1
        
        if(i1 == word_len):    #porównywanie ciągów zostało wykonane pozytywnie po wszystkich znakach
            break
        
        result -= 1
    
    return result


#funkcja zwraca maksymalną liczbę nukleotydów, na których zazębia się sufiks nucl1 z prefiksem nucl2
#poszukiwanie poprzez coraz to mniejszą długość ciągu, bezpośrednie porównywanie ciągów przy pomocy mechanizmu "slicing"
def common_nucleotides_max_number_iterating_with_slicing(nucl1, nucl2):
    word_len = len(nucl1)
    result = word_len - 1       #zmienna zawierająca końcowy wynik
    
    while(result > 0):          #badanie coraz to mniejszej możliwej długości wspólnego ciągu
        i1 = word_len - result
        i2 = 0
        
        if(nucl1[-result:] == nucl2[:result]): #porównywanie ciągów przy pomocy mechanizmu "slicing"
            break
        
        result -= 1
    
    return result

In [ ]:
#---------------- Testowanie szybkości wyznaczania wspólego sufiksu nucl1 z prefiksem nucl2 ----------------#
def speed_test_common_nucleotides_max_number(function, repetition):
    print("Function testing in progress...")
    start_time = time.time()
    filename = negative_random_filenames[0]
    with open(filename, 'r') as file:
        inst_name = filename[filename.rfind('/') + 1:]

        words = file.read().splitlines()

        word_len = len(words[0])
        org_words_num = int(re.search('(?<=\.)[0-9]+', inst_name).group(0))
        org_seq_len = org_words_num + word_len - 1
        neg_errors_number = re.search('(?<=\-)[0-9]+', inst_name)
        neg_errors_number = int(neg_errors_number.group(0)) if neg_errors_number else 0
        pos_errors_number = re.search('(?<=\+)[0-9]+', inst_name)
        pos_errors_number = int(pos_errors_number.group(0)) if pos_errors_number else 0
        words_num = len(words)
        
        while(repetition > 0):
            for i in range(words_num):
                for j in range(words_num):
                    if(i != j):
                        number = function(words[i], words[j])
            repetition -= 1
    
    elapsed_time = time.time() - start_time
    print("End of the function testing.")
    print("Time: " + str(elapsed_time) + "\n")

#Wywołanie testowania funkcji
#speed_test_common_nucleotides_max_number(common_nucleotides_max_number_full_iterating, 10)            #repetition=10 Time: 13.98816466331482
#speed_test_common_nucleotides_max_number(common_nucleotides_max_number_iterating_with_slicing, 10)    #repetition=10 Time: 16.22281813621521

In [ ]:
def set_starting_wages(n, k):
    result = np.zeros((n))
    n -= 1
    result[n] = 1.0
    while(n > 0):
        n -= 1
        result[n] = result[n + 1] / k
    
    return result

#funkcja oblicza początkowe wagi prawdopodobieństwa dla wyboru j-tego wyrazu po dokonaniu już wyboru i-tego
#waga przyznawana jest na podstawie długości wspólnego sufiksu-prefiksu słowa i-tego i j-tego
def generate_next_word_power_matrix_1():
    #przydzielenia wag dla konkretnych połączeń wyrazów
    for i in range(1, words_num + 1):
        for j in range(1, words_num + 1):
            next_word_power_matrix[i, j] = wages[ common_nucleotides_matrix[i, j] ]

#funkcja oblicza początkowe wagi prawdopodobieństwa dla wyboru j-tego wyrazu po dokonaniu już wyboru i-tego
#określona już waga prawdopodobieństwa dla połączenia wyrazów o pewnej długości zazębiających się końcówek jest równomiernie rozdzielana na wsystkie połączenia o tejże długości wspólnego ciągu
def generate_next_word_power_matrix_2():
    for i in range(1, words_num + 1):
        #zliczenie wystąpień zachodzenia wyrazów na danej liczbie znaków
        occurrence_counter = np.zeros(word_len)
        for j in range(1, words_num + 1):
            occurrence_counter[ common_nucleotides_matrix[i, j] ] += 1
        
        #obliczenie wartości wag prawdopodobieństwa obowiązujących dla danego i-tego wiersza -> dla poszukiwania kolejnego wyrazu po i-tym wyrazie
        current_wages = np.copy(wages)
        for j in range(word_len):
            if(occurrence_counter[j] != 0):
                current_wages[j] /= occurrence_counter[j]
        
        #przydzielenia wag dla konkretnych połączeń wyrazów
        for j in range(1, words_num + 1):
            next_word_power_matrix[i, j] = current_wages[ common_nucleotides_matrix[i, j] ]
    

<hr />
<h2>Wyznaczenie wag prawdopodobieństwa dla wyboru wyrazu rozpoczynającego sekwencje</h2>

In [ ]:
#funkcja ustawia równe szanse wyboru słowa rozpoczynającego sekwencje
def generate_chance_for_first_word():
    next_word_power_matrix[0, 1:] = 1;

#funkcja ustawia większe szanse wyboru słowa dla wyrazów, których prefiks w maksymalnym przypadku pokrywa się na jaknajmniejszej ilości zanków z jakim kolwiek sufiksem
def generate_chance_for_first_word_1_full_iterating():
    max_common_nucleotides = np.zeros(words_num + 1, dtype=int)

    #wyznaczenie maksymalnej długości zazębiającego się prefiksu j-tego wyrazu
    for j in range(1, words_num + 1):
        max_common_nucleotides[j] = common_nucleotides_matrix[1, j]
        for i in range(2, words_num + 1):
            if(max_common_nucleotides[j] < common_nucleotides_matrix[i, j]):
                max_common_nucleotides[j] = common_nucleotides_matrix[i, j]
    
    #przyznanie wagi prawdopodobieństwa przy użyciu ustalonych wag
    for j in range(1, words_num + 1):
        next_word_power_matrix[0, j] = wages[ word_len - 1 - max_common_nucleotides[j] ]

        
#funkcja ustawia większe szanse wyboru słowa dla wyrazów, których prefiks w maksymalnym przypadku pokrywa się na jaknajmniejszej ilości zanków z jakim kolwiek sufiksem
#szybsza wersja wykorzystująca możliwości biblioteki numpy
def generate_chance_for_first_word_1_with_numpy_tricks():
    #wyznaczenie maksymalnej długości zazębiającego się prefiksu j-tego wyrazu
    max_common_nucleotides = common_nucleotides_matrix.max(0)
    
    #przyznanie wagi prawdopodobieństwa przy użyciu ustalonych wag
    next_word_power_matrix[0] = wages[ word_len - 1 - max_common_nucleotides]
    
    

#funkcja ustawia większe szanse wyboru słowa dla wyrazów, których prefiks w maksymalnym przypadku pokrywa się na jaknajmniejszej ilości zanków z jakim kolwiek sufiksem
#określona już waga jest równomiernie rozdzielona na wyrazy o poszczególnej długości maksymalnego zazębienia
def generate_chance_for_first_word_2_full_iterating():
    max_common_nucleotides = np.zeros(words_num + 1, dtype=int)

    #wyznaczenie maksymalnej długości zazębiającego się prefiksu j-tego wyrazu
    for j in range(1, words_num + 1):
        max_common_nucleotides[j] = common_nucleotides_matrix[1, j]
        for i in range(2, words_num + 1):
            if(max_common_nucleotides[j] < common_nucleotides_matrix[i, j]):
                max_common_nucleotides[j] = common_nucleotides_matrix[i, j]
    
    #przyznanie wagi prawdopodobieństwa przy użyciu ustalonych wag, waga jest równomiernie rozdizelona
    occurrence_counter = np.zeros(word_len)
    for j in range(1, words_num + 1):
        occurrence_counter[ common_nucleotides_matrix[0, j] ] += 1

    #obliczenie wartości wag prawdopodobieństwa obowiązujących dla danego i-tego wiersza -> dla poszukiwania kolejnego wyrazu po i-tym wyrazie
    current_wages = np.copy(wages)
    for j in range(word_len):
        if(occurrence_counter[j] != 0):
            current_wages[j] /= occurrence_counter[j]

    #przydzielenia wag dla konkretnych połączeń wyrazów
    for j in range(1, words_num + 1):
        next_word_power_matrix[0, j] = current_wages[ word_len - 1 - max_common_nucleotides[j] ]
        
        
#funkcja ustawia większe szanse wyboru słowa dla wyrazów, których prefiks w maksymalnym przypadku pokrywa się na jaknajmniejszej ilości zanków z jakim kolwiek sufiksem
#określona już waga jest równomiernie rozdzielona na wyrazy o poszczególnej długości maksymalnego zazębienia
#szybsza wersja wykorzystująca możliwości biblioteki numpy
def generate_chance_for_first_word_2_with_numpy_tricks():
    #wyznaczenie maksymalnej długości zazębiającego się prefiksu j-tego wyrazu
    max_common_nucleotides = common_nucleotides_matrix.max(0)
    
    #przyznanie wagi prawdopodobieństwa przy użyciu ustalonych wag, waga jest równomiernie rozdizelona
    occurrence_counter = np.zeros(word_len)
    unique, counts = np.unique(common_nucleotides_matrix[0, 1:], return_counts=True)
    occurrence_counter[unique] = counts

    #obliczenie wartości wag prawdopodobieństwa obowiązujących dla danego i-tego wiersza -> dla poszukiwania kolejnego wyrazu po i-tym wyrazie
    current_wages = np.copy(wages)
    occurrence_counter[occurrence_counter == 0] = 1
    current_wages /= occurrence_counter

    #przydzielenia wag dla konkretnych połączeń wyrazów
    next_word_power_matrix[0] = current_wages[ word_len - 1 - max_common_nucleotides]

In [ ]:
#---------------- Testowanie szybkości wyznaczania wag prawdopodobień dla wyboru pierwszego wyrazu w sekwencji ----------------#
def speed_test_generate_chance_for_first_word(function, repetition):
    print("Preparing required variables...")
    filename = negative_random_filenames[0]
    with open(filename, 'r') as file:
        inst_name = filename[filename.rfind('/') + 1:]

        words = file.read().splitlines()

        word_len = len(words[0])
        org_words_num = int(re.search('(?<=\.)[0-9]+', inst_name).group(0))
        org_seq_len = org_words_num + word_len - 1
        neg_errors_number = re.search('(?<=\-)[0-9]+', inst_name)
        neg_errors_number = int(neg_errors_number.group(0)) if neg_errors_number else 0
        pos_errors_number = re.search('(?<=\+)[0-9]+', inst_name)
        pos_errors_number = int(pos_errors_number.group(0)) if pos_errors_number else 0
        words_num = len(words)

        #dodanie pozornego słowa w celu zaimplementowania sytuacji wyboru pierwszego wyrazu rozpoczynającego sekwencję
        words = ["Poczatek"] + words


        #wyznaczenie macierzy mówiącej o długości wspólnego i-tego sufiksu z j-tym prefiksem
        common_nucleotides_matrix = np.zeros((words_num + 1, words_num + 1), dtype=int)
        for i in range(1, words_num + 1):
            for j in range(1, words_num + 1):
                if(i != j):
                    number = common_nucleotides_max_number_full_iterating(words[i], words[j])
                    common_nucleotides_matrix[i, j] = number
                else:
                    common_nucleotides_matrix[i, i] = 0

        wages = set_starting_wages(word_len, 2)


        next_word_power_matrix = np.zeros((words_num + 1, words_num + 1))
        generate_next_word_power_matrix_2()
        
        
        print("Function testing in progress...")
        start_time = time.time()
        while(repetition > 0):
            function()
            repetition -= 1


        elapsed_time = time.time() - start_time
        print("End of the function testing.")
        print("Time: " + str(elapsed_time) + "\n\n")

#Wywołanie testowania funkcji
#speed_test_generate_chance_for_first_word(generate_chance_for_first_word, 100)                   #repetition=100 Time: 0.0
#speed_test_generate_chance_for_first_word(generate_chance_for_first_word_1_full_iterating, 100)    #repetition=100 Time: 7.25950026512146
#speed_test_generate_chance_for_first_word(generate_chance_for_first_word_2_full_iterating, 100)    #repetition=100 Time: 7.367282152175903
#speed_test_generate_chance_for_first_word(generate_chance_for_first_word_1_with_numpy_tricks, 100) #repetition=100 Time: 0.020961999893188477
speed_test_generate_chance_for_first_word(generate_chance_for_first_word_2_with_numpy_tricks, 100)  #repetition=100 Time: 0.026450395584106445

In [ ]:
#---------------- Sprawdzanie poprawności wyznaczania wag prawdopodobień dla wyboru pierwszego wyrazu w sekwencji ----------------#
print("Preparing required variables...")
filename = negative_random_filenames[0]
with open(filename, 'r') as file:
    inst_name = filename[filename.rfind('/') + 1:]

    words = file.read().splitlines()

    word_len = len(words[0])
    org_words_num = int(re.search('(?<=\.)[0-9]+', inst_name).group(0))
    org_seq_len = org_words_num + word_len - 1
    neg_errors_number = re.search('(?<=\-)[0-9]+', inst_name)
    neg_errors_number = int(neg_errors_number.group(0)) if neg_errors_number else 0
    pos_errors_number = re.search('(?<=\+)[0-9]+', inst_name)
    pos_errors_number = int(pos_errors_number.group(0)) if pos_errors_number else 0
    words_num = len(words)

    #dodanie pozornego słowa w celu zaimplementowania sytuacji wyboru pierwszego wyrazu rozpoczynającego sekwencję
    words = ["Poczatek"] + words


    #wyznaczenie macierzy mówiącej o długości wspólnego i-tego sufiksu z j-tym prefiksem
    common_nucleotides_matrix = np.zeros((words_num + 1, words_num + 1), dtype=int)
    for i in range(1, words_num + 1):
        for j in range(1, words_num + 1):
            if(i != j):
                number = common_nucleotides_max_number_full_iterating(words[i], words[j])
                common_nucleotides_matrix[i, j] = number
            else:
                common_nucleotides_matrix[i, i] = 0

    wages = set_starting_wages(word_len, 2)


    next_word_power_matrix = np.zeros((words_num + 1, words_num + 1))
    generate_next_word_power_matrix_2()


    print("Function testing in progress...")
    #generate_chance_for_first_word_1_full_iterating()
    generate_chance_for_first_word_2_full_iterating()
    vector1 = np.copy(next_word_power_matrix[0])
    #generate_chance_for_first_word_1_with_numpy_tricks()
    generate_chance_for_first_word_2_with_numpy_tricks()
    vector2 = np.copy(next_word_power_matrix[0])
    
    print(vector2 - vector1)


    print("End of the function testing.")

<hr />

In [ ]:
#zmienne
avaible_words_number = 0
#max_sum = 0

In [ ]:
#funkcja wybiera kolejny wyraz dla tworzonego ciągu
def choose_next_word():
    global avaible_words_number
    #global max_sum
    #print("avaible words przed: " + str(avaible_words_number))
    #wybranie odpowiednich wektorów z macierzy, które przechowują dane dla ostatnio wybranego słowa
    common_nucleotides_vector = common_nucleotides_matrix[current_word]
    next_word_power_vector = next_word_power_matrix[current_word]
    
    #jeśli obecna długość wynikowego ciągu zwiększona o pełną długość kolejnego słowa przekracza ograniczającą długość ciągu znaków,
    #to trzeba wykluczyć możliwość wyboru kolejnego słowa, które by to za bardzo zwiększyła długość wynikowej sekwencji
    #print(str(current_length) + " + " + str(word_len) + " > " + str(org_seq_len))
    if(current_length + word_len > org_seq_len):
        for j in range(1, words_num + 1):
            if((avaible_words[j]) and (current_length + (word_len - common_nucleotides_vector[j]) > org_seq_len)):
                avaible_words[j] = 0
                avaible_words_number -= 1
        
        #print("Juz prawie koniec - mozna dopasowac jeszcze " + str(avaible_words_number) + " slow")
    
    #nie ma już dostępnych słów do doklejenia
    if(not(avaible_words_number)):
        return -1
    
    #obliczenie sumy mocy prawdopodobieństwa wyboru dla dostępnych słów
    sum_of_probability = 0
    for j in range(1, words_num + 1):
        if(avaible_words[j]):
            sum_of_probability += next_word_power_vector[j]

            
    """#print(str(max_sum) + " < " + str(sum_of_probability))
    if(max_sum < sum_of_probability):
        max_sum = sum_of_probability
        print("sum_of_probability = " + str(sum_of_probability))"""
    
    #realizacja wyboru zgodnie z gęstością prawdopodobieństwa - wybór punktu na odcinku złożonego z sumy mocy prawdopodobieństwa dostępnych połączeń
    point_in_range_of_sum_of_probability = random.uniform(0, sum_of_probability)
    
    #odnalezienie wybranego kolejnego słowa, które zostało wskazane przez wylosowany punkt na odcinku
    j = 1
    while(not(avaible_words[j])):
        j += 1
    
    current_sum_of_probability = next_word_power_vector[j]
    while(point_in_range_of_sum_of_probability > current_sum_of_probability):
        j += 1
        while(not(avaible_words[j])):
            j += 1
        current_sum_of_probability += next_word_power_vector[j]
    
    #print("avaible words po: " + str(avaible_words_number))
    
    return j
    

In [ ]:
#funkcja dokonuje wygładzania wartości wag prawdopodobieństw -> zmienia szanse wyboru na bardziej równe, zachowująć jednocześnie porządek preferencji
def compensation_of_next_word_power_matrix():
    compensation_coef = 0.98
    base_of_logarithm = 2
    
    for i in range(words_num + 1):
        max_value = max(next_word_power_matrix[i])
        sum_value = sum(next_word_power_matrix[i])
        
        #jeśli maksymalna wartość zdecydowanie się wyróżnia, to należy złagodzić dysproporcje między wagami prawdopodobieństw
        if(max_value > sum_value * compensation_coef):
            
            #poszukiwanie minimalnej wartości wagi prawdopodobieństwa nie mniejszej niż wartość 1. Wartość ta jest potrzebna do wzoru wygładzającego
            min_value = max_value
            for j in range(1, words_num + 1):
                if((next_word_power_matrix[i, j] >= 1) and (min_value > next_word_power_matrix[i, j])):
                    min_value = next_word_power_matrix[i, j]
            
            #wygładzanie wag większych niż wartość 1
            for j in range(1, words_num + 1):
                if(min_value < next_word_power_matrix[i, j]):
                    next_word_power_matrix[i, j] = min_value * (1 + math.log(next_word_power_matrix[i, j] / min_value, base_of_logarithm))

<hr/>
<h1>Główny kod wykonujący obliczenia zgodnie z algorytmem mrówkowym</h1>

In [ ]:
filename = negative_random_filenames[0]
with open(filename, 'r') as file:
    inst_name = filename[filename.rfind('/') + 1:]

    words = file.read().splitlines()

    word_len = len(words[0])
    org_words_num = int(re.search('(?<=\.)[0-9]+', inst_name).group(0))
    org_seq_len = org_words_num + word_len - 1
    neg_errors_number = re.search('(?<=\-)[0-9]+', inst_name)
    neg_errors_number = int(neg_errors_number.group(0)) if neg_errors_number else 0
    pos_errors_number = re.search('(?<=\+)[0-9]+', inst_name)
    pos_errors_number = int(pos_errors_number.group(0)) if pos_errors_number else 0
    words_num = len(words)

    print("nazwa pliku: " + inst_name)
    print("długość słowa: " + str(word_len))
    print("oryginalna liczba słów: " + str(org_words_num))
    print("oryginalna długość sekwencji: "+ str(org_seq_len))
    print("liczba błędów negatywnych: " + str(neg_errors_number))
    print("liczba błędów pozytywnych: " + str(pos_errors_number))
    print("liczba oligonukleotydów: " + str(words_num))
    
    #dodanie pozornego słowa w celu zaimplementowania sytuacji wyboru pierwszego wyrazu rozpoczynającego sekwencję
    words = ["Poczatek"] + words
    print(words[:5])
    print("\n")
    
    
    #wyznaczenie macierzy mówiącej o długości wspólnego i-tego sufiksu z j-tym prefiksem
    print("common_nucleotides_matrix")
    common_nucleotides_matrix = np.zeros((words_num + 1, words_num + 1), dtype=int)
    for i in range(1, words_num + 1):
        for j in range(1, words_num + 1):
            if(i != j):
                number = common_nucleotides_max_number_full_iterating(words[i], words[j])
                #print(words[0])
                #print((word_len - number) * ' ' + words[i] + "\tnumber=" + str(number) + "\n")
                common_nucleotides_matrix[i, j] = number
            else:
                common_nucleotides_matrix[i, i] = 0
    #print(common_nucleotides_matrix[0, :])
    
    print("set_starting_wages()")
    wages = set_starting_wages(word_len, 2)
    #print(wages)
    
    
    #wyznaczanie macierzy mocy prawdopodobieństwa następienia j-tego wyrazu po i-tym
    next_word_power_matrix = np.zeros((words_num + 1, words_num + 1))
    print("generate_next_word_power_matrix_1()")
    generate_next_word_power_matrix_1()
    #print("generate_next_word_power_matrix_2()")
    #generate_next_word_power_matrix_2()
    
    #ustawienie wag prawdopodobieństwa dla rozpoczęcia tworzenia sekwencji przez dany wyraz
    #print("generate_chance_for_first_word()")
    #generate_chance_for_first_word()
    print("generate_chance_for_first_word_1_with_numpy_tricks()")
    generate_chance_for_first_word_1_with_numpy_tricks()
    #print("generate_chance_for_first_word_2_with_numpy_tricks()")
    #generate_chance_for_first_word_2_with_numpy_tricks()
    
    
    iterations_number = 100    #definicja zmiennej określającej ile iteracji będzie trwał algorytm mrówkowy
    #max_w_macierzy = 0
    
    #wykonywanie poszczególnej iteracji; wykonywanie pętli tyle razy, ile jest zadeklarowanych iteracji do wykonania
    for indeks_of_iteration in range(iterations_number):
        attempts_number = 100  #definicja zmiennej określającej ile razy dokonywane jest budowanie sekwencji w jednej iteracji
        sequences = []
        
        
        
        #tworzenie wynikowej sekwencji; powtarzanie prób uzyskania rozwiązania tyle razy, ile jest to określone parametrem 'attempts_number'
        print("\nIteracja " + str(indeks_of_iteration) + ":\t\nTrwa wykonywanie wszystkich " + str(attempts_number) + " prób utworzenia wynikowej sekwencji.")
        for indeks_of_attempt in range(attempts_number):
            current_length = 0                         #definicja zmiennej zawierającej długość utworzonej dotychczas sekwencji
            avaible_words_number = words_num           #definicja zmiennej zawierającej liczbę niewykorzystanych jeszcze wyrazów
            avaible_words = np.ones((words_num + 1))   #definicja wektora określającego, czy i-ty wyraz jest dostępny
            sequence = []
            
            avaible_words[0] = 0        #odznaczenie użycia już zerowego słowa ('X')
            current_word = 0            #definicja zmiennej wskazującej ostatnio wybranego słowa -> słowo zerowe, czyli zaczynane jest tworzenie sekwencji
            sequence.append(current_word)
            
            #nieustannie wykonująca się pętla - zostanie przerwana, gdy nie będzie już możliwości doczepienia słowa przy ograniczeniu ilości znaków w wynikowym ciągu
            while(True):
                next_word = choose_next_word()
                #print("Wybrane slowo: " + str(next_word))
                
                if(next_word == -1):
                    break
                
                current_length += word_len - common_nucleotides_matrix[current_word, next_word]
                avaible_words_number -= 1
                avaible_words[next_word]= 0
                
                """print(words[current_word])
                print((word_len - common_nucleotides_matrix[current_word, next_word]) * ' ' + words[next_word] + "\tPokrywanie na tylu znakach: " + str(common_nucleotides_matrix[current_word, next_word]))
                print("Liczba niewykorzystanych jeszcze slow: " + str(avaible_words_number))
                print("Obecna dlugosc ciagu: " + str(current_length))
                print("\n")"""
                
                current_word = next_word
                sequence.append(current_word)
            
            #print("Koniec tworzenia ciagu.")
            sequences.append(sequence)
            
        print("Wykonano wszystkie sekwencje w obecnej iteracji")
        """for i in range(len(sequences)):
            print(str(i) + ":\t" + str(len(sequences[i])))"""
            
        sequences.sort(key = lambda x: len(x), reverse=True)

        """for i in range(len(sequences)):
            print(str(i) + ":\t" + str(len(sequences[i])))"""
        
        #print(sequences[0])
        #print(sequences[1])
        #nagrodzenie najlepszych rozwiązań
        part_of_the_best_sequences = 0.1
        amount_of_the_best_sequences = int(attempts_number * part_of_the_best_sequences)
        for i in range(amount_of_the_best_sequences):
            print(str(i) + ":\t" + str(len(sequences[i])))
        best_price_for_sequence = 1.2
        for iSequence in range(amount_of_the_best_sequences):
            sequence = sequences[iSequence]
            price_for_sequence = 1 + (best_price_for_sequence - 1) * ((amount_of_the_best_sequences - iSequence) / amount_of_the_best_sequences)
            #print("price_for_sequence = " + str(price_for_sequence))
            for iConnection in range(len(sequence) - 1):
                first_word = sequence[iConnection]
                second_word = sequence[iConnection + 1]
                
                price_for_connection = 1 + (price_for_sequence - 1) * (common_nucleotides_matrix[first_word, second_word] / (word_len - 1))
                next_word_power_matrix[first_word, second_word] *= price_for_connection
                
                """#print(str(max_sum) + " < " + str(sum_of_probability))
                if(max_w_macierzy < next_word_power_matrix[first_word, second_word]):
                    max_w_macierzy = next_word_power_matrix[first_word, second_word]
                    print("max_w_macierzy = " + str(max_w_macierzy))
                    print("nagradzalem mnozac przez " + str(price_for_connection))
                    print("price_for_sequence = " + str(price_for_sequence))"""
            
            
            """wyswietlanie mnożnika dla połączeń - im bardziej wyrazy się zazębiają, tym większa liniowo jest wartość mnożnika
            for i in range(word_len):
                price_for_connection = 1 + (price_for_sequence - 1) * (i / (word_len - 1))
                print("price_for_connection = " + str(price_for_connection))"""
    
    
        #wygładzanie macierzy wag prawdopodobieństwa w celu zapobiegnięcia nadmiernemu zdominowaniu przez najlepsze rozwiązanie
        print("wykonywanie funkcji compensation_of_next_word_power_matrix...")
        compensation_of_next_word_power_matrix()
    
    print("koniec")

<hr /><hr /><br /><br /><br />
<h3>Poniżej można swobodnie testować i sprawdzać działanie wykorzystywanych konstrukcji języka python.</h3>

In [ ]:
avaible_words_number = 0
def testF():
    print(testVar)
    print("testVar2 = " + str(testVar2))

In [ ]:
testVar = 5
testF()
for i in range(5):
    testVar2 = i
    testF()

In [ ]:
for i in range(20):
    print(random.uniform(0.9992892359651135, 0.9992892359651136))

In [ ]:
def testPrzekazywanie(x):
    x += 10

x = 5
print(x)
testPrzekazywanie(x)
print(x)

In [ ]:
vartest = np.array([[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12]])
print(vartest)
print(vartest.shape)
print(vartest[1])
print(max(vartest[1]))

In [ ]:
print(math.log(625, 5))

In [ ]:
x = np.zeros(6)
print(x)
y = np.zeros((4))
print(y)
z = np.zeros(13, dtype=int)
print(z)

In [ ]:
"""#wyznaczenie maksymalnej długości zazębiającego się prefiksu j-tego wyrazu
for j in range(1, words_num + 1):
    max_common_nucleotides[j] = common_nucleotides_matrix[1, j]
    for i in range(2, words_num + 1):
        if(max_common_nucleotides[j] < common_nucleotides_matrix[i, j]):
            max_common_nucleotides[j] = common_nucleotides_matrix[i, j]"""

x = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
print(x)
y = x.max(0)
print(y)



In [ ]:
"""#przyznanie wagi prawdopodobieństwa przy użyciu ustalonych wag
    for j in range(1, words_num + 1):
        next_word_power_matrix[0, j] = wages[ word_len - 1 - max_common_nucleotides[j] ]"""


a = np.array([1, 2, 3, 5, 6])
print(a)
b = np.array([0, 1, 2, 1, 0])
print(b)
values = np.array([5, 20, 100])
print(values)

a = values[2 - b]
print(a)


In [ ]:
"""occurrence_counter = np.zeros(word_len)
for j in range(1, words_num + 1):
    occurrence_counter[ common_nucleotides_matrix[0, j] ] += 1"""

x = np.array([0, 1,3, 2, 1, 1, 1, 4, 6, 6, 2])
y = np.zeros(7, dtype=int)
unique, counts = np.unique(x, return_counts=True)
y[unique] = counts
print(y)


In [ ]:
"""#obliczenie wartości wag prawdopodobieństwa obowiązujących dla danego i-tego wiersza -> dla poszukiwania kolejnego wyrazu po i-tym wyrazie
    current_wages = np.copy(wages)
    for j in range(word_len):
        if(occurrence_counter[j] != 0):
            current_wages[j] /= occurrence_counter[j]"""

x = np.array([1, 4, 9, 4, 10, 30])
y = np.array([1, 2, 3, 0, 2, 5])

print(y)
y[y == 0] = 1
print(y)

z = x / y
#z = np.where(y != 0, x / y, x)
#z = np.where(y != 0, 'a', 'b')
print(z)